## Initialize the client

In [1]:
import crowdcent_challenge as cc
import polars as pl
import numpy as np


client = cc.ChallengeClient(
    challenge_slug="hyperliquid-ranking",
    api_key="hD5EB4pf.KvY8281Kz4igdNBCK7P0RL6tG6J8EaAU",
    base_url="http://localhost:8080/api",
)

2025-05-07 02:08:00,665 - INFO - ChallengeClient initialized for 'hyperliquid-ranking' at URL: http://localhost:8080/api


## Get challenge and training datasets details

In [2]:
client.get_challenge()

{'id': 6,
 'name': 'Hyperliquid Ranking',
 'slug': 'hyperliquid-ranking',
 'description': 'A demo challenge for ranking crypto assets.',
 'start_date': '2025-03-08T01:15:43.315966Z',
 'end_date': None,
 'is_active': True}

In [3]:
client.list_training_datasets()

[]

## Download current inference data

In [4]:
client.download_inference_data(
    release_date="current", dest_path="inference_data.parquet"
)

2025-05-07 02:08:00,706 - INFO - Downloading inference data for challenge 'hyperliquid-ranking' current to inference_data.parquet
2025-05-07 02:08:00,724 - INFO - Successfully downloaded inference data to inference_data.parquet


In [5]:
# Read in the inference data, in this case, just a universe
inference_data = pl.read_parquet("inference_data.parquet")
inference_data.head()

id
str
"""AAVE"""
"""ACE"""
"""ADA"""
"""AI"""
"""AI16Z"""


## Submit predictions

In [6]:
# Add prediction columns with random float values for demonstration
# Replace this with your actual prediction logic

inference_data = inference_data.with_columns(
    [
        pl.Series("pred_10d", np.random.random(len(inference_data))).cast(pl.Float64),
        pl.Series("pred_30d", np.random.random(len(inference_data))).cast(pl.Float64),
    ]
)

# Display the first few rows to verify the new columns
print("Inference data with prediction columns:")
print(inference_data.head())

# You can save this modified dataframe if needed
inference_data.write_parquet("inference_data_with_predictions.parquet")

Inference data with prediction columns:
shape: (5, 3)
┌───────┬──────────┬──────────┐
│ id    ┆ pred_10d ┆ pred_30d │
│ ---   ┆ ---      ┆ ---      │
│ str   ┆ f64      ┆ f64      │
╞═══════╪══════════╪══════════╡
│ AAVE  ┆ 0.952141 ┆ 0.887545 │
│ ACE   ┆ 0.8793   ┆ 0.573542 │
│ ADA   ┆ 0.197027 ┆ 0.546677 │
│ AI    ┆ 0.429971 ┆ 0.666005 │
│ AI16Z ┆ 0.557173 ┆ 0.099974 │
└───────┴──────────┴──────────┘


In [7]:
client.submit_predictions(file_path="inference_data_with_predictions.parquet", slot=1)

2025-05-07 02:08:00,770 - INFO - Submitting predictions from inference_data_with_predictions.parquet to challenge 'hyperliquid-ranking' (Slot: 1)
2025-05-07 02:08:00,837 - INFO - Successfully submitted predictions to challenge 'hyperliquid-ranking'


{'id': 1479,
 'username': 'admin',
 'challenge_slug': 'hyperliquid-ranking',
 'inference_data_release_date': '2025-05-07T02:07:50.696076Z',
 'submitted_at': '2025-05-07T02:08:00.819449Z',
 'status': 'pending',
 'slot': 1,
 'score_details': None,
 'prediction_file': 'http://localhost:8080/media/challenges/hyperliquid-ranking/submissions/1/716/inference_data_with_predictions.parquet'}

In [8]:
for slot in range(1, 6):
    client.submit_predictions(
        file_path="inference_data_with_predictions.parquet", slot=slot
    )

2025-05-07 02:08:00,851 - INFO - Submitting predictions from inference_data_with_predictions.parquet to challenge 'hyperliquid-ranking' (Slot: 1)
2025-05-07 02:08:00,895 - INFO - Successfully submitted predictions to challenge 'hyperliquid-ranking'
2025-05-07 02:08:00,896 - INFO - Submitting predictions from inference_data_with_predictions.parquet to challenge 'hyperliquid-ranking' (Slot: 2)
2025-05-07 02:08:00,935 - INFO - Successfully submitted predictions to challenge 'hyperliquid-ranking'
2025-05-07 02:08:00,936 - INFO - Submitting predictions from inference_data_with_predictions.parquet to challenge 'hyperliquid-ranking' (Slot: 3)
2025-05-07 02:08:01,019 - INFO - Successfully submitted predictions to challenge 'hyperliquid-ranking'
2025-05-07 02:08:01,020 - INFO - Submitting predictions from inference_data_with_predictions.parquet to challenge 'hyperliquid-ranking' (Slot: 4)
2025-05-07 02:08:01,058 - INFO - Successfully submitted predictions to challenge 'hyperliquid-ranking'
2025

## Meta-model

In [9]:
client.download_meta_model(dest_path="meta_model.parquet")
pl.read_parquet("meta_model.parquet").head()

2025-05-07 02:08:01,124 - INFO - Downloading consolidated meta model for challenge 'hyperliquid-ranking' to meta_model.parquet
2025-05-07 02:08:02,687 - INFO - Successfully downloaded consolidated meta model to meta_model.parquet


id,pred_10d,pred_30d,release_date
str,f64,f64,date
"""LDO""",-0.096787,0.077537,2025-04-07
"""AI""",-0.202331,0.089412,2025-04-07
"""ZRO""",-0.025278,-0.145994,2025-04-07
"""kLUNC""",0.078609,0.113463,2025-04-07
"""PANDORA""",0.191133,0.082821,2025-04-07
